In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
import lightgbm as lgb

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 80)

import gc

In [2]:
def cv_score(df_train, y_train, kfolds, pipeline, imp_coef=False):
    oof = np.zeros([y_train.shape[0], y_train.shape[1]])
    
    train = df_train.copy()
    
    feat_df = pd.DataFrame()
    
    for n_fold, (train_index, test_index) in enumerate(kfolds.split(train.values)):
            
        trn_data = train.iloc[train_index][:]
        val_data = train.iloc[test_index][:]
        
        trn_target = y_train[train_index]
        val_target = y_train[test_index]
        
        pipeline.fit(trn_data, trn_target)

        oof[test_index, :] = pipeline.predict(val_data)

        if imp_coef:
            try:
                fold_df = get_coef(pipeline)
            except AttributeError:
                fold_df = get_feature_importance(pipeline)
                
            fold_df['fold'] = n_fold + 1
            feat_df = pd.concat([feat_df, fold_df], axis=0)
       
    if imp_coef:
        feat_df = feat_df.groupby('feat')['score'].agg(['mean', 'std'])
        feat_df['abs_sco'] = (abs(feat_df['mean']))
        feat_df = feat_df.sort_values(by=['abs_sco'],ascending=False)
        del feat_df['abs_sco']
        return oof, feat_df
    else:    
        return oof

In [3]:
def get_coef(pipe):
    imp = pipe.steps[-1][1].coef_.tolist()
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result['abs_res'] = abs(result['score'])
    result = result.sort_values(by=['abs_res'],ascending=False)
    del result['abs_res']
    return result


def get_feature_importance(pipe):
    imp = pipe.steps[-1][1].feature_importances_.tolist() #it's a pipeline
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result = result.sort_values(by=['score'],ascending=False)
    return result

def _plot_diagonal(ax):
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    low = min(xmin, xmax)
    high = max(xmin, xmax)
    scl = (high - low) / 100
    
    line = pd.DataFrame({'x': np.arange(low, high ,scl), # small hack for a diagonal line
                         'y': np.arange(low, high ,scl)})
    ax.plot(line.x, line.y, color='black', linestyle='--')
    
    return ax


def plot_predictions(data, true_label, pred_label, feature=None, hue=None, legend=False, savename='test.png'):
    '''
    Plot prediction vs true label or a specific feature. It also plots the residuals plot
    '''
    
    tmp = data.copy()
    tmp['Prediction'] = pred_label
    tmp['True Label'] = true_label
    tmp['Residual'] = tmp['True Label'] - tmp['Prediction']
    
    diag = False
    alpha = 0.7
    label = ''
    
    fig, ax = plt.subplots(1,2, figsize=(15,6))
    
    if feature is None:
        feature = 'True Label'
        diag = True
    else:
        legend = 'full'
        sns.scatterplot(x=feature, y='True Label', data=tmp, ax=ax[0], label='True',
                         hue=hue, legend=legend, alpha=alpha)
        label = 'Predicted'
        alpha = 0.4

    sns.scatterplot(x=feature, y='Prediction', data=tmp, ax=ax[0], label=label,
                         hue=hue, legend=legend, alpha=alpha)
    if diag:
        ax[0] = _plot_diagonal(ax[0])
    
    sns.scatterplot(x=feature, y='Residual', data=tmp, ax=ax[1], 
                    hue=hue, legend=legend, alpha=0.7)
    ax[1].axhline(y=0, color='r', linestyle='--')
    
    ax[0].set_title(f'{feature} vs Predictions')
    ax[1].set_title(f'{feature} vs Residuals')
    
    if not savename.endswith('.png'):
        savename += '.png'
    plt.savefig('../plots/' + savename)
    plt.close()

In [4]:
class df_imputer(TransformerMixin, BaseEstimator):
    '''
    Just a wrapper for the SimpleImputer that keeps the dataframe structure
    '''
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.imp = None
        self.statistics_ = None

    def fit(self, X, y=None):
        self.imp = SimpleImputer(strategy=self.strategy)
        self.imp.fit(X)
        self.statistics_ = pd.Series(self.imp.statistics_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Ximp = self.imp.transform(X)
        Xfilled = pd.DataFrame(Ximp, index=X.index, columns=X.columns)
        return Xfilled

    
class df_scaler(TransformerMixin, BaseEstimator):
    '''
    Wrapper of StandardScaler or RobustScaler
    '''
    def __init__(self, method='standard'):
        self.scl = None
        self.scale_ = None
        self.method = method
        if self.method == 'sdandard':
            self.mean_ = None
        elif method == 'robust':
            self.center_ = None
        self.columns = None  # this is useful when it is the last step of a pipeline before the model

    def fit(self, X, y=None):
        if self.method == 'standard':
            self.scl = StandardScaler()
            self.scl.fit(X)
            self.mean_ = pd.Series(self.scl.mean_, index=X.columns)
        elif self.method == 'robust':
            self.scl = RobustScaler()
            self.scl.fit(X)
            self.center_ = pd.Series(self.scl.center_, index=X.columns)
        self.scale_ = pd.Series(self.scl.scale_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xscl = self.scl.transform(X)
        Xscaled = pd.DataFrame(Xscl, index=X.index, columns=X.columns)
        self.columns = X.columns
        return Xscaled

    def get_feature_names(self):
        return list(self.columns)


class dummify(TransformerMixin, BaseEstimator):
    '''
    Wrapper for get dummies
    Via match_cols, it is possible to ask the transformer to make sure that all the dummies are there
    Missing dummies are introduced with a column of 0's
    Extra dummies are dropped
    '''
    def __init__(self, drop_first=False, match_cols=True):
        self.drop_first = drop_first
        self.columns = []  # useful to well behave with FeatureUnion
        self.match_cols = match_cols

    def fit(self, X, y=None):
        return self
    
    def match_columns(self, X):
        miss_train = list(set(X.columns) - set(self.columns))
        miss_test = list(set(self.columns) - set(X.columns))
        
        err = 0
        
        if len(miss_test) > 0:
            for col in miss_test:
                X[col] = 0  # insert a column for the missing dummy
                err += 1
        if len(miss_train) > 0:
            for col in miss_train:
                del X[col]  # delete the column of the extra dummy
                err += 1
                
        if err > 0:
            warnings.warn('The dummies in this set do not match the ones in the train set, we corrected the issue.',
                         UserWarning)
            
        return X[self.columns]
        
        
    def transform(self, X):
        X = pd.get_dummies(X, drop_first=self.drop_first)
        if (len(self.columns) > 0):
            if self.match_cols:
                X = self.match_columns(X)
        else:
            self.columns = X.columns
        return X
    
    def get_features_name(self):
        return list(self.columns)

In [5]:
class transformation(TransformerMixin, BaseEstimator):
    def __init__(self, mean_weight=10):
        self.columns = None

        
    def fit(self, X, y=None):
        return self
    
    
    def stats_by_play(self, data):
        avg_by_play = data.groupby(['PlayId', 
                                    'Team', 
                                    'offense_team'], as_index=False)[['PlayerHeight', 
                                                                      'PlayerWeight',
                                                                      'age',
                                                                      'S', 'A']].mean()
        spread = data.groupby(['PlayId', 
                               'Team', 
                               'offense_team'])[['X', 'Y']].std().reset_index()
        tot_momentum = data.groupby(['PlayId', 
                                     'Team', 
                                     'offense_team'], as_index=False)[['X_speed', 'Y_speed',
                                                                       'PlayerWeight',
                                                                       'X_acceleration','Y_acceleration']].sum()
        
        tot_momentum['x_momentum'] = tot_momentum['X_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['y_momentum'] = tot_momentum['Y_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['x_force'] = tot_momentum['X_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum['y_force'] = tot_momentum['Y_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum.drop(['X_speed', 'Y_speed',
                           'PlayerWeight',  
                           'X_acceleration','Y_acceleration'], axis=1, inplace=True)

        avg_by_play = pd.merge(avg_by_play, tot_momentum, on=['PlayId', 'Team', 'offense_team'])
        avg_by_play = pd.merge(avg_by_play, spread, on=['PlayId', 'Team', 'offense_team'])

        poss_team = avg_by_play[avg_by_play.Team == avg_by_play.offense_team].copy()
        def_team = avg_by_play[avg_by_play.Team != avg_by_play.offense_team].copy()

        poss_team.rename(columns={'PlayerHeight': 'poss_avg_height', 
                                  'PlayerWeight': 'poss_avg_weight',
                                  'age': 'poss_avg_age',
                                  'X': 'poss_std_X',
                                  'Y': 'poss_std_Y',
                                  'S': 'poss_avg_S', 
                                  'A': 'poss_avg_A', 
                                  'x_momentum': 'poss_x_momentum', 
                                  'y_momentum': 'poss_y_momentum', 
                                  'x_force': 'poss_x_force', 
                                  'y_force': 'poss_y_force'}, inplace=True)
        def_team.rename(columns={'PlayerHeight': 'def_avg_height', 
                                  'PlayerWeight': 'def_avg_weight', 
                                  'age': 'def_avg_age',
                                  'X': 'def_std_X',
                                  'Y': 'def_std_Y',
                                  'S': 'def_avg_S', 
                                  'A': 'def_avg_A',
                                  'x_momentum': 'def_x_momentum', 
                                  'y_momentum': 'def_y_momentum', 
                                  'x_force': 'def_x_force', 
                                  'y_force': 'def_y_force'}, inplace=True)

        avg_by_play = pd.merge(poss_team.drop('Team', axis=1), 
                               def_team.drop('Team', axis=1), on=['PlayId', 'offense_team'])
        
        avg_by_play['tot_x_momenumt'] = avg_by_play['poss_x_momentum'] - avg_by_play['def_x_momentum']
        avg_by_play['tot_x_force'] = avg_by_play['poss_x_force'] - avg_by_play['def_x_force']
        avg_by_play['height_diff'] = avg_by_play['poss_avg_height'] - avg_by_play['def_avg_height']
        avg_by_play['weight_diff'] = avg_by_play['poss_avg_weight'] - avg_by_play['def_avg_weight']
        avg_by_play['age_diff'] = avg_by_play['poss_avg_age'] - avg_by_play['def_avg_age']
        avg_by_play['X_diff'] = avg_by_play['poss_std_X'] - avg_by_play['def_std_X']
        avg_by_play['Y_diff'] = avg_by_play['poss_std_Y'] - avg_by_play['def_std_Y']

        return avg_by_play
    
    
    def process_play(self, X):
        cols_by_play = ['GameId', 'PlayId', 'YardLine', 
                'Quarter', 'GameClock', 'Down', 'Distance',
                'OffenseFormation', 'DefendersInTheBox',  
                'Location', 'StadiumType', 'Turf', 
                'GameWeather','Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 
                'PlayDirection', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay']
        train_play = X[cols_by_play].drop_duplicates()
        avg_by_play = self.stats_by_play(X)
        train_play = pd.merge(train_play, avg_by_play.drop('offense_team', axis=1), on=['PlayId'])

        return train_play
    
    def get_team(self, data):
        data.loc[:, 'Team_offense'] = np.where(data.offense_team == 'home', 
                                               data.HomeTeamAbbr, data.VisitorTeamAbbr)
        data.loc[:, 'Team_defense'] = np.where(data.offense_team == 'home', 
                                               data.VisitorTeamAbbr, data.HomeTeamAbbr)
        data.loc[data.offense_team == 'home', 'is_home'] = 1
        data.loc[data.offense_team == 'away', 'is_home'] = 0
    
        return data
    
    
    def dropper(self, X):
        cleaned = X.copy()
        to_drop = ['GameId', 'WindDirection', 'WindSpeed', 'GameWeather', 
                    'PlayDirection', 'StadiumType', 'Location', 
                   'GameClock', 'distance_from_ball', 'Quarter', 'Down', 
                   'OffenseFormation', 'Temperature', 'Humidity', 
                   'HomeScoreBeforePlay','VisitorScoreBeforePlay', 'offense_team', 
                   'HomeTeamAbbr', 'VisitorTeamAbbr',
                   'Dis', 'Dir', 'Yards', 'Distance', 'PlayId', 'X', 'Y']
        for col in to_drop:
            try:
                del cleaned[col]
            except KeyError:
                pass
        
        return cleaned
    
    
    def transform(self, X, y=None):
        train_play = self.process_play(X)
        carriers = X[X.has_ball].copy()

        to_drop = ['GameId', 'NflId', 'Team', 'Orientation','YardLine', 'Quarter', 'GameClock', 'PossessionTeam',
           'Down', 'FieldPosition', 'HomeScoreBeforePlay',
           'VisitorScoreBeforePlay', 'NflIdRusher', 'OffensePersonnel','DefensePersonnel',
               'PlayDirection', 'Position', 
           'Location', 'StadiumType', 'GameWeather',
           'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'to_left',
           'has_ball', 'Distance',
           'OffenseFormation', 'DefendersInTheBox', 'Turf']    

        carriers.drop(to_drop, axis=1, inplace=True)

        full_train = pd.merge(carriers, train_play, on='PlayId')
        
        full_train = self.get_team(full_train)
        
        full_train = self.dropper(full_train)
        
        self.columns = full_train.columns

        return full_train
    
    
    def get_features_name(self):
        return self.columns

In [6]:
def create_targets(data):
    unique_plays = data[['PlayId', 'Yards']].drop_duplicates()
    yards = unique_plays['Yards'].reset_index(drop=True)
    
    y = np.zeros((yards.shape[0], 199))
    for idx, target in enumerate(list(yards)):
        y[idx][99 + target] = 1
    
    return yards, y


def crps_score(y_prediction, y_valid, shape):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

In [7]:
df_train = pd.read_csv('../data_processed/train_processed.csv', dtype={'WindSpeed': 'object'})

df_train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,Yards,PlayerHeight,PlayerWeight,Position,HomeTeamAbbr,VisitorTeamAbbr,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,to_left,has_ball,offense_team,from_yardline,X_speed,Y_speed,X_acceleration,Y_acceleration,age,distance_from_ball,closest_opponent,opponents_in_6,teammates_in_6
0,2017090700,20170907000118,away,46.09,18.493333,1.69,1.13,0.40,81.99,1.620015,496723,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,72,212,SS,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,1.09,1.687953,-0.083145,1.128632,-0.055594,10480,6.480872,4.59331,3.0,7.0
1,2017090700,20170907000118,away,45.33,20.693333,0.42,1.35,0.01,27.61,1.244420,2495116,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,288,DE,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,0.33,0.397828,0.134657,1.278734,0.432828,10394,4.593310,4.59331,3.0,7.0
2,2017090700,20170907000118,away,46.00,20.133333,1.22,0.59,0.31,3.01,1.174083,2495493,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,270,DE,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,1.00,1.125250,0.471395,0.544178,0.227970,10457,5.448982,4.59331,3.0,7.0
3,2017090700,20170907000118,away,48.54,25.633333,0.42,0.54,0.02,359.77,2.868623,2506353,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,245,ILB,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,3.54,0.113229,-0.404449,0.145580,-0.520006,12709,7.820038,4.59331,3.0,7.0
4,2017090700,20170907000118,away,50.68,17.913333,1.82,2.43,0.16,12.63,1.844638,2530794,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,72,206,FS,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,5.68,1.752185,-0.492187,2.339456,-0.657151,10980,10.622476,4.59331,3.0,7.0


In [8]:
yards, y = create_targets(df_train)

In [9]:
transf_pipe = Pipeline([('trsf', transformation())])

full_train = transf_pipe.fit_transform(df_train)

full_train.head()

,S,A,PlayerHeight,PlayerWeight,from_yardline,X_speed,Y_speed,X_acceleration,Y_acceleration,age,closest_opponent,opponents_in_6,teammates_in_6,YardLine,DefendersInTheBox,Turf,poss_avg_height,poss_avg_weight,poss_avg_age,poss_avg_S,poss_avg_A,poss_x_momentum,poss_y_momentum,poss_x_force,poss_y_force,poss_std_X,poss_std_Y,def_avg_height,def_avg_weight,def_avg_age,def_avg_S,def_avg_A,def_x_momentum,def_y_momentum,def_x_force,def_y_force,def_std_X,def_std_Y,tot_x_momenumt,tot_x_force,height_diff,weight_diff,age_diff,X_diff,Y_diff,Team_offense,Team_defense,is_home
0,3.63,3.35,70,205,3.75,1.491487,3.309436,1.376441,3.054163,9349,4.593310,3.0,7.0,45,6.0,0,74.727273,259.181818,10374.454545,2.106364,1.358182,30394.065885,42824.323203,14808.310917,29092.727988,1.772665,5.855606,73.636364,233.545455,10101.272727,1.314545,1.025455,21967.405111,4867.003640,11406.041625,-783.137540,5.294079,7.014714,8426.660774,3402.269293,1.090909,25.636364,273.181818,-3.521414,-1.159107,NE,KC,1.0
1,3.06,2.41,70,205,4.07,-2.055465,2.266862,-1.618847,1.785339,9349,4.287773,3.0,7.0,53,6.0,0,74.727273,259.181818,10374.454545,2.094545,1.541818,5361.669172,36553.979512,1503.788380,27039.089576,2.001380,6.932502,73.636364,233.545455,10101.272727,1.639091,1.592727,-3627.891830,6256.141431,-2706.292685,4533.856018,5.406292,7.190716,8989.561002,4210.081065,1.090909,25.636364,273.181818,-3.404912,-0.258215,NE,KC,1.0
2,5.77,2.42,70,205,3.66,4.290640,3.857889,1.799541,1.618040,9349,4.221670,2.0,6.0,75,7.0,0,74.727273,259.181818,10374.454545,3.682727,1.419091,86277.195695,48107.583525,33664.932926,23232.912519,2.115286,6.044208,73.636364,233.545455,10101.272727,3.244545,2.092727,82977.152392,12860.033647,52837.531067,8496.045083,4.720893,6.577620,3300.043303,-19172.598141,1.090909,25.636364,273.181818,-2.605606,-0.533412,NE,KC,1.0
3,4.45,3.20,71,210,3.53,-0.421875,4.429957,-0.303371,3.185587,9808,4.528002,7.0,9.0,108,9.0,0,76.181818,282.545455,10320.636364,2.141818,0.880909,-39812.029305,39225.794933,-11636.814745,16186.365888,1.898686,3.154869,73.727273,257.454545,10390.181818,1.555455,1.293636,-37637.593645,-4639.830146,-26960.759277,-9328.539063,0.962418,4.505029,-2174.435660,15323.944532,2.454545,25.090909,-69.545455,0.936268,-1.350160,NE,KC,1.0
4,3.90,2.53,71,216,5.01,-3.613974,1.466013,-2.344450,0.951029,8069,4.288088,3.0,6.0,35,7.0,0,76.909091,268.454545,9732.818182,2.644545,1.620000,-31427.161922,20004.068984,-13921.201825,13577.076191,2.056883,7.423977,73.181818,242.454545,9967.181818,2.322727,2.121818,-54049.085285,-11151.985380,-50459.233839,-5737.783040,5.391251,7.864325,22621.923362,36538.032014,3.727273,26.000000,-234.363636,-3.334368,-0.440348,KC,NE,0.0


In [32]:
class feat_eng(BaseEstimator, TransformerMixin):
    def __init__(self, mean_weight=1000, te_team_mean=True):
        self.mean_tot = 0
        self.std_tot = 0
        self.mean_weight = mean_weight
        self.smooth_off = {}
        self.smooth_def = {}
        self.te_team_mean = te_team_mean
    
    
    def smooth_te(self, data, target, col):
        tmp_data = data.copy()
        tmp_data['target'] = target.argmax(axis=1) - 99
        mean_tot = tmp_data['target'].mean()
        means = tmp_data.groupby(col)['target'].mean()
        counts = tmp_data.groupby(col)['target'].count()

        smooth = ((counts * means + self.mean_weight * mean_tot) / 
                       (counts + self.mean_weight))
        return mean_tot, smooth
    
    
    def fit(self, X, y):
        if self.te_team_mean:
            self.mean_tot, self.smooth_off = self.smooth_te(data=X, target=y, col='Team_offense')
            self.mean_tot, self.smooth_def = self.smooth_te(data=X, target=y, col='Team_defense')

        return self
    
   
    def tr_teams(self, data):
        if self.te_team_mean:
            data['avg_gained'] = data['Team_offense'].map(self.smooth_off).fillna(self.mean_tot)
            data['avg_lost'] = data['Team_defense'].map(self.smooth_def).fillna(self.mean_tot)
            data['exp_gained'] = (data['avg_gained'] + data['avg_lost']) / 2
            
        del data['Team_offense']
        del data['Team_defense']

        return data
    
    def speed_at_yrdl(self, data):
        data['speed_at_yrdl'] = data['X_speed'] + np.sqrt(2 * data['A'] * data['from_yardline'])
        return data
    
    def ages_years(self, data):
        cols = [col for col in data.columns if 'age' in col]
        for col in cols:
            data[col] = data[col] // 365
        return data
    
    def wei_hei(self, data):
        data['wei_hei'] = data['PlayerWeight'] / data['PlayerHeight']
        return data
    
    def ostility(self, data):
        data['opp_ratio'] = data['opponents_in_6'] / data['teammates_in_6']
        data['opp_ratio'].replace(np.inf, 0, inplace=True)
        return data
    
    def drop(self, data):
        to_drop = ['weight_diff', 'def_x_momentum', 'pos_x_momentum', 'avg_gained',
                   'def_avg_height', 'pos_avg_height', 'def_avg_S', 'age', 'age_diff',
                   'opponents_in_6', 'teammates_in_6']
        for col in to_drop:
            try:
                del data[col]
            except KeyError:
                pass
        return data
    
    
    def transform(self, X, y=None):
        X = self.tr_teams(X)
        X = self.speed_at_yrdl(X)
        X = self.ages_years(X)
        X = self.wei_hei(X)
        X = self.ostility(X)
        X = self.drop(X)
        
        return X

In [33]:
folds = KFold(5, shuffle=True, random_state=541)

plays = df_train[df_train.has_ball][['PlayId', 
                                     'YardLine', 
                                     'from_yardline', 
                                     'X']].drop_duplicates().reset_index(drop=True)

In [35]:
forest_pipe = Pipeline([('fe', feat_eng()),
                        ('dummifier', dummify(drop_first=True)),
                        ('Imputer', df_imputer()),
                        ('scl', df_scaler(method='robust')),
                        ('forest', RandomForestRegressor(n_estimators=200, n_jobs=-1,  
                                               criterion='mse', max_features=0.8, random_state=651))])
train = full_train.copy()

%time inf_preds, imps= cv_score(train, y, folds, forest_pipe, imp_coef=True)

crps = crps_score(inf_preds, y, train.shape[0])
    
inf_preds = inf_preds.argmax(axis=1) - 99

mae = mean_absolute_error(yards, inf_preds)

print(round(crps, 5), round(mae, 5))

imps

CPU times: user 52min 59s, sys: 4.13 s, total: 53min 4s
Wall time: 9min 24s
0.01288 3.57697


,mean,std
feat,,
YardLine,0.032871,0.000368
Y_acceleration,0.031273,0.000403
def_y_momentum,0.030681,0.000164
from_yardline,0.029871,0.000110
A,0.029801,0.000250
closest_opponent,0.029202,0.000126
Y_diff,0.029175,0.000192
Y_speed,0.029154,0.000199
poss_std_X,0.029044,0.000199


In [17]:
to_train = []

crps_prev = 99
mae_prev = 99
bad = []

tmp = full_train.copy()
tmp = feat_eng().fit_transform(tmp, y)

for col in imps.index:
    print(col)
    forest_pipe = Pipeline([
                        ('dummifier', dummify(drop_first=True)),
                        ('Imputer', df_imputer()),
                        ('scl', df_scaler(method='robust')),
                        ('forest', RandomForestRegressor(n_estimators=100, n_jobs=-1,  
                                              criterion='mse', max_features='auto', random_state=651))])
    try:
        to_train.append(col)
        train = tmp[to_train].copy()
    except KeyError:
        to_train.remove(col)
        continue

    inf_preds= cv_score(train, y, folds, forest_pipe)
    
    crps = crps_score(inf_preds, y, train.shape[0])
    
    inf_preds = inf_preds.argmax(axis=1) - 99

    mae = mean_absolute_error(yards, inf_preds)

    print(len(to_train), round(crps, 5), round(mae, 5))
    
    crps_diff = crps_prev - crps
    mae_diff = mae_prev - mae
    
    if crps_diff <= 0:
        if mae_diff < 0:
            bad.append(col)
            print('_'*40)
            print(f'{crps_diff} - {mae_diff}')
            print('_'*40)
    
    crps_prev = crps
    mae_prev = mae
    
    del train
    gc.collect()

YardLine
1 0.01395 3.82137
Y_acceleration
2 0.01622 5.00052
________________________________________
-0.0022720000000000014 - -1.1791463467265113
________________________________________
def_y_momentum
3 0.01445 4.45686
from_yardline
4 0.01393 4.14117
A
5 0.01356 3.9694
Y_speed
6 0.01336 3.88499
poss_std_X
7 0.01327 3.8089
Y_diff
8 0.01319 3.75469
closest_opponent
9 0.01316 3.73907
def_avg_A
10 0.0131 3.71417
def_y_force
11 0.01308 3.69203
poss_y_momentum
12 0.01307 3.6872
poss_avg_A
13 0.01305 3.68745
tot_x_force
14 0.01305 3.66713
tot_x_momenumt
15 0.01304 3.65099
poss_y_force
16 0.01304 3.64969
def_avg_S
17 0.01305 3.65207
________________________________________
-4.000000000000531e-06 - -0.0023736567260801422
________________________________________
def_avg_weight
18 0.01303 3.63895
def_std_X
19 0.01298 3.64305
speed_at_yrdl
20 0.01292 3.62319
weight_diff
21 0.01293 3.61741
poss_avg_weight
22 0.01292 3.63113
poss_avg_S
23 0.01292 3.62837
X_diff
24 0.01294 3.62453
def_std_Y
25 0.012

In [18]:
bad

['Y_acceleration',
 'def_avg_S',
 'def_avg_height',
 'poss_avg_height',
 'def_x_momentum',
 'avg_gained',
 'age',
 'max_lost',
 'max_gained',
 'age_diff',
 'opponents_in_6']